# 1. Create population

#### 1.1 Import customer list from CSV

In [10]:
import pandas as pd

# path to customers list
csv_file_path = 'customers.csv'

# create pandas dataframe with all customers from CSV-file
CUSTOMERS_DF = pd.read_csv(csv_file_path)
print("CSV loaded successfully. Number of customers: ", CUSTOMERS_DF.shape[0])
print("First 5 customers: ", CUSTOMERS_DF.head(5))

CSV loaded successfully. Number of customers:  50
First 5 customers:     id          x          y
0   1  63.942680   2.501076
1   2  27.502932  22.321074
2   3  73.647121  67.669949
3   4  89.217957   8.693883
4   5  42.192182   2.979722


#### 1.2 Select random population of 14 customers

In [38]:
"""
This section selects 14 random customers from the pandas dataframe.
random_state is set to 42 to ensure reproducibility.
"""
POPULATION_SIZE = 14

# select customers randomly from customers_list_df to match needed population size.
customer_population = customers_list_df.sample(n=POPULATION_SIZE, random_state=42)
print("First 5 customers:")
print(customer_population.head(5))

# list of customer ids of selected customers
customer_ids = customer_population['id'].tolist()
print("----------------------------")
print("List of ids of selected customers:", customer_ids)

First 5 customers:
    id          x          y
13  14   9.274584   9.671638
39  40  26.488017  24.662751
30  31  98.952335  63.999976
45  46  10.964913  62.744604
17  18  37.853438  55.204063
----------------------------
List of ids of selected customers: [14, 40, 31, 46, 18, 49, 27, 26, 33, 20, 13, 5, 38, 9]


#### 1.3 Define generations, depot and vehicles.

In [39]:
# number of generations and vehicles
GENERATIONS = 3
VEHICLES = 3

# coordinated for depot
DEPOT = (0,0)

# marker of depot it chromosome list
DEPOT_MARKER = 0

# list of internal depot markers, with the constraint that all vehicles start and finish at depot (0)
num_internal_depot_markers = VEHICLES - 1

#### 1.4 Create initial random chromosome.

In [49]:
import random

"""
Constraints:
1. start at depot (0,0)
2. 3 vehicles. start and end at depot. e.g. (0,...,...,...,0,...,...,0,...,...,...,...,0)
3. each customer visited only once
"""

# create random first chromosome
def create_individual():
    """
    Creates the first random individual chromosome based on the customers string and the number of vehicles.
    """

    # create list of internal depot markers
    internal_depot_markers_list = [depot_marker] * num_internal_depot_markers

    # combine list of internal_depot_markers and customer_ids
    elements_to_shuffle = internal_depot_markers_list + customer_ids

    # shuffle all customers and internal_depot_markers randomly
    random.shuffle(elements_to_shuffle)

    # create chromosome which start at 0, and end at 0, with the customers and random point of depot-visits inbetween
    chromosome = [depot_marker] + elements_to_shuffle + [depot_marker]

    print(chromosome)

create_individual()



[0, 38, 26, 27, 14, 0, 13, 0, 31, 18, 9, 33, 20, 46, 40, 5, 49, 0]


# 2. Fitness function